In [1]:
import numpy as np
import pandas as pd
from IPython.display import display
import matplotlib.pyplot as plt



In [22]:
pred = pd.read_csv('pred_logis_val.csv')
display(pred)

,Unnamed: 0,0,1
0,0,0.993936,0.006064
1,1,0.997234,0.002766
2,2,0.997471,0.002529
3,3,0.996463,0.003537
4,4,0.998621,0.001379
5,5,0.994543,0.005457
6,6,0.989999,0.010001
7,7,0.985641,0.014359
8,8,0.997304,0.002696
9,9,0.999231,0.000769


In [23]:
print("confidence interval for the ones:[%6.4f - %6.4f]" % (np.percentile(pred['1'],2.5),np.percentile(pred['1'],97.5)))
print("confidence interval for the zeros:[%6.4f - %6.4f]" % (np.percentile(pred['0'],2.5),np.percentile(pred['0'],97.5)))

confidence interval for the ones:[0.0001 - 0.0493]
confidence interval for the zeros:[0.9507 - 0.9999]


In [24]:
ratio = len(pred['0'])/len(pred['1'])

In [25]:
print(ratio)

1.0


# strategy

In [11]:
row_train = pd.read_csv('train.csv', sep=',')
row_val = pd.read_csv('validation.csv', sep=',')
row_test = pd.read_csv('test.csv', sep=',')
bidprice = pd.read_csv('xgb169bidpricevalidation.csv', sep=',')
bidprice.bidprice = bidprice.bidprice 
bidprice[bidprice['bidprice']>300]=300

In [3]:
def bad_ctr(x):
    new = pd.crosstab(index=x, columns=row_train['click'])
    new['Clicks']=new[1]
    new['Imps'] = pd.crosstab(index=x, columns='counts')

    
#CTR: Click-Through Rate = #clicks / #imps advertise   (per Advertiser)
    new['CTR']=(new[1]/new['Imps'])*100
    new['best']=new.CTR>=0.015
#hours.CTR.value_counts().plot(kind='bar')
    #display(new)
    #indexes = [index for index in (new) if new['best'][index] == False]
    indexes= new[new['best']==False]
    return indexes.index.values


In [7]:
print(bidprice.describe())

            bidprice
count  303375.000000
mean       94.494719
std       222.153105
min         3.363698
25%        33.587112
50%        52.628132
75%        84.495707
max      6537.708953


In [95]:
bad_cities = bad_ctr(row_train['city'])
#bad_days = bad_ctr(row_train['weekday'])
bad_hours = bad_ctr(row_train['hour'])
bad_region = bad_ctr(row_train['region'])
bad_slot_width = bad_ctr(row_train['slotwidth'])
bad_slot_height = bad_ctr(row_train['slotheight'])
print(bad_slot_height)

[100 150 230]


In [96]:
#bidprice.bidprice =bidprice.bidprice+15
index =[]

def evaluate(bidprice):
    budget=6250000
    impression=0
    click=0
    for row in range(0,row_val.shape[0]):
        if budget<row_val.payprice[row]:
            continue
        if row_val.city[row] not in bad_cities:
            if row_val.hour[row] not in bad_hours:
                if row_val.region[row] not in bad_region:
                    if row_val.slotwidth[row] not in bad_slot_width:
                        if row_val.slotheight[row] not in bad_slot_height:
                            if bidprice.bidprice[row]>row_val.payprice[row]:
                                budget=budget-row_val.payprice[row]
                                impression=impression+1
                                if row_val.click[row]==1:
                                    click=click+1
                                    index.append(row)
    print ('budget',budget)                                
    return click

new_clicks = evaluate(bidprice)
print(new_clicks)
print(index)

budget 101282
152
[891, 1702, 2136, 2292, 3061, 3253, 5834, 8447, 11612, 12233, 13226, 14760, 18603, 18705, 27357, 29027, 30837, 33097, 36216, 38982, 42362, 43266, 43394, 44921, 45288, 45965, 45969, 46461, 47173, 47437, 47595, 52849, 53373, 54628, 61027, 61318, 63139, 65536, 67869, 72779, 78527, 79032, 81131, 83115, 88339, 89557, 91655, 92445, 93670, 96394, 103275, 103339, 103460, 107994, 115206, 115582, 116879, 118415, 119970, 121238, 121757, 122242, 124585, 125941, 130387, 131163, 135188, 141368, 144729, 145122, 145344, 147665, 149188, 150444, 154381, 158332, 159811, 160562, 165542, 166431, 169721, 169832, 170040, 172066, 174002, 174255, 177270, 180149, 183045, 187176, 191202, 193820, 194228, 194761, 195638, 195890, 196287, 198351, 199281, 200587, 202293, 203850, 208642, 210446, 210534, 212414, 215056, 215509, 216471, 217090, 217629, 218782, 227945, 230661, 231614, 234043, 236964, 240749, 241708, 242694, 247138, 250031, 251605, 253915, 254272, 258199, 259954, 263502, 268555, 271411, 

In [ ]:
#158, 193565,0.1 
#158,208431 ,0.15
#+5, 143, 0
#+3, 152, 0
#+2, 158, 1251
#+2,    #tope 250

In [12]:
index=[]
def evaluate2(bidprice):
    budget=6250000
    impression=0
    click=0
    
    for row in range(0,row_val.shape[0]):
        if budget<row_val.payprice[row]:
            continue

        if bidprice.bidprice[row]>row_val.payprice[row]:
            budget=budget-row_val.payprice[row]
            impression=impression+1
            if row_val.click[row]==1:
                click=click+1
                index.append(row)
    print ('budget',budget)                                
    return click

new_clicks = evaluate2(bidprice)
print(new_clicks)
print(index)
#159, budget 36046, price limit 262
#   , price limit 300

budget 0
160
[891, 1702, 2136, 2292, 3061, 3253, 5834, 8447, 11612, 12233, 13226, 14760, 18603, 18705, 27357, 29027, 30837, 33097, 36216, 38982, 42362, 43266, 43394, 44921, 45288, 45965, 45969, 46461, 47173, 47437, 47595, 52782, 52849, 52955, 53373, 54628, 61027, 61318, 63139, 65536, 67869, 72779, 78527, 79032, 81131, 83115, 88339, 89557, 91655, 92445, 93670, 96394, 103275, 103339, 103460, 107994, 113500, 115206, 115582, 116879, 118415, 119970, 121238, 121757, 122242, 124585, 124913, 125941, 130387, 131163, 135188, 137008, 138680, 141368, 144729, 145122, 145344, 147665, 149188, 150444, 154381, 158332, 159811, 160562, 165542, 166431, 169721, 169832, 170040, 172066, 174002, 174255, 176881, 177270, 180149, 183045, 187176, 191202, 193820, 194228, 194761, 195638, 195890, 196287, 198351, 199281, 199297, 200587, 202293, 203850, 208642, 210446, 210534, 212414, 215056, 215509, 216471, 217090, 217629, 218782, 227945, 230661, 231614, 232697, 234043, 236964, 240749, 241708, 242694, 247138, 247622,

In [97]:
no_click = row_val[row_val['click']==1]

display(no_click)

,click,weekday,hour,bidid,userid,useragent,IP,region,city,adexchange,...,slotheight,slotvisibility,slotformat,slotprice,creative,bidprice,payprice,keypage,advertiser,usertag
891,1,2,21,0690d908f6e5f361c41b03c47add534406dbfcfa,611b62b2f86fc3ce2ef70a11a3e184197bc023e5,mac_safari,58.47.13.*,201,208,1.0,...,600,2,1,0,cb7c76e7784031272e37af8e7e9b062c,300,51,bebefa5efe83beee17a3d245e7c5085b,1458,"11278,10111"
1702,1,0,19,74adda6dba283a4f0d88ef6b33f5717ca435fac6,fa9c9107cde0bdfe6b0ff1bdd90c821b08e7fe71,windows_chrome,114.244.206.*,1,1,3.0,...,250,0,0,50,a499988a822facd86dd0e8e4ffef8532,300,50,bebefa5efe83beee17a3d245e7c5085b,1458,"13866,11278,10006,10063,10111"
2136,1,5,15,5f76f428776cafe69f4730f3572cc9e62db0388b,647a2483259a8cabae5b93d8403b89aa35018d95,android_other,119.115.225.*,40,47,1.0,...,250,1,1,0,911b2d84826786018761e8c0b0a3a60c,227,97,d29e59bf0f7f8243858b8183f14d4412,3358,10684
2292,1,0,23,8e6bda5e84e7a4aea20f3cae8e78b5243ecce355,258ee0a34f6173301cbe819ff7662c82dbf9619a,windows_ie,121.20.20.*,3,5,3.0,...,250,0,0,50,44966cc8da1ed40c95d59e863c8c75f0,300,50,43f4a37f42a7c5e6219e2601b26c6976,3386,NaN
3061,1,2,19,f7f63a06e23135bfae8b0ae3f44bfe90c74ccf50,1ebf6feef86667dadb0f9e251e5c44bbfa03c6b1,android_safari,114.138.88.*,298,299,NaN,...,50,FirstView,Na,6,11908,277,48,NaN,2997,NaN
3253,1,1,19,b9361a7122bb2410d51c580a55c315f51a169c2c,ef2dbeb568be7df4be80c8eb1e972631e2f56503,windows_chrome,125.39.9.*,2,2,3.0,...,90,0,0,70,832b91d59d0cb5731431653204a76c0e,300,70,bebefa5efe83beee17a3d245e7c5085b,1458,"10006,11278,10063,10077,10111"
5834,1,4,21,99d18b3d4a795e8e7262f332fec844ca2a1fdc23,df57c6a8b84bd814dfd67383537c5e3bc0472c7d,android_safari,182.147.205.*,276,294,NaN,...,50,OtherView,Na,5,12987,277,31,NaN,2997,NaN
8447,1,3,16,c66f985984270b1c937163f31d1a888b30575ed5,3f9cc77b4e48b926f7839d81ce25cbfe26c9e1ea,mac_safari,120.192.246.*,333,336,1.0,...,90,Na,Na,0,12628,294,61,NaN,2261,NaN
11612,1,2,13,b53ed20cceb458a6ddbb37c67e0222e8d23a2f00,a8a1aa3adbaea301bbd5732054581c0d7a5941e3,windows_ie,125.85.123.*,275,275,2.0,...,90,1,0,5,7184c9560e68e977187e67e45a4f3198,238,122,0f951a030abdaedd733ee8d114ce2944,3427,"13042,10006,13776,10110"
12233,1,5,10,bb2c6a9da6adb082ab40a2ae249f021bddeb991e,204fd686d4ead2c5f1d4e65545c904eb608f072f,mac_safari,123.101.245.*,164,165,1.0,...,250,0,1,0,44966cc8da1ed40c95d59e863c8c75f0,300,112,361e128affece850342293213691a043,3386,"10052,13866,10111"


In [50]:
display(bidprice[bidprice['bidprice']>150])

,bidid,bidprice
18,db12485ab1e09236b00aa8de2f19cac829711996,167.179330
33,c136e27c82b523f19fa061cb4882485cf1cf3b6e,156.012846
40,ded61a53baf71f81935fd2e8c99ecbcd5f783f1a,364.859056
47,3a37037477b74366a22b5697c5e71375efe880b0,516.524077
64,05c453bf19716c5d94f52397b66d53173bc2e9c2,335.606842
68,80b3fac4d107b44470d8f1452d69c367d73ca11a,439.112553
86,551736b0be0e5e84f7af2c00c8592bf17af939fa,214.782759
103,69907166a266e81bc8be86ef76ef9e630bdb9188,1400.095217
106,c1e465ffd4900cf5ea02de878a2059624658a88a,943.791657
124,f14ad086e78045d27151a8a3a094e4320e649ca6,657.530580
